In [1]:
import joblib

In [2]:
# load motion data
data_path = "ACCAD_Male2MartialArtsStances_c3d_D5 - ready to walk away_poses.pt"
adam_pose = joblib.load(data_path)

In [3]:
adam_pose['root_vel'][0]

array([-0.03629386,  0.00692368,  0.03256917], dtype=float32)

In [27]:
def amass2adam(useful_poses):

    num_frames = useful_poses.shape[0]

    pelvis = useful_poses[:,0,:]

    l_upperarm = useful_poses[:,16,:]
    l_forearm = useful_poses[:,18,:]
    l_hand = useful_poses[:,20,:]

    r_upperarm = useful_poses[:,17,:]
    r_forearm = useful_poses[:,19,:]
    r_hand = useful_poses[:,21,:]

    l_thigh = useful_poses[:,1,:]
    l_calf = useful_poses[:,4,:]
    l_foot = useful_poses[:,7,:]
    l_toe = useful_poses[:,10,:]

    r_thigh = useful_poses[:,2,:]
    r_calf = useful_poses[:,5,:]
    r_foot = useful_poses[:,8,:]
    r_toe = useful_poses[:,11,:]

    # upper body
    head = (l_upperarm + r_upperarm) / 2
    upper_z = (head - pelvis) / np.linalg.norm(head - pelvis, axis=1, keepdims=True)
    upper_x = np.cross(pelvis - l_upperarm, pelvis - r_upperarm)
    upper_x = upper_x / np.linalg.norm(upper_x, axis=1, keepdims=True)
    upper_y = np.cross(upper_z, upper_x)
    upper_rot = np.stack([upper_x, upper_y, upper_z], axis=2)

    # lower body
    crotch = (l_thigh + r_thigh) / 2
    lower_z = - (crotch - pelvis) / np.linalg.norm(crotch - pelvis, axis=1, keepdims=True)
    lower_x = np.cross(pelvis - r_thigh, pelvis - l_thigh)
    lower_x = lower_x / np.linalg.norm(lower_x, axis=1, keepdims=True)
    lower_y = np.cross(lower_z, lower_x)
    lower_rot = np.stack([lower_x, lower_y, lower_z], axis=2)

    # left elbow
    l_elbow_rot_z = l_forearm - l_hand
    l_elbow_rot_z = l_elbow_rot_z / np.linalg.norm(
        l_elbow_rot_z, axis=1, keepdims=True
    )

    l_elbow_rot_y = l_upperarm - l_forearm
    l_elbow_rot_y = l_elbow_rot_y / np.linalg.norm(
        l_elbow_rot_y, axis=1, keepdims=True
    )
    l_elbow_rot_y = np.cross(l_elbow_rot_y, -l_elbow_rot_z)
    l_elbow_rot_y = l_elbow_rot_y / np.linalg.norm(
        l_elbow_rot_y, axis=1, keepdims=True
    )

    l_elbow_rot_x = np.cross(l_elbow_rot_y, l_elbow_rot_z)
    l_elbow_rot_x = l_elbow_rot_x / np.linalg.norm(
        l_elbow_rot_x, axis=1, keepdims=True
    )

    l_elbow_rot = np.stack([l_elbow_rot_x, l_elbow_rot_y, l_elbow_rot_z], axis=2)

    l_upperarm_dir = l_forearm - l_upperarm
    l_upperarm_dir = l_upperarm_dir / np.linalg.norm(
        l_upperarm_dir, axis=1, keepdims=True
    )

    # right elbow
    r_elbow_rot_z = r_forearm - r_hand
    r_elbow_rot_z = r_elbow_rot_z / np.linalg.norm(
        r_elbow_rot_z, axis=1, keepdims=True
    )

    r_elbow_rot_y = r_upperarm - r_forearm
    r_elbow_rot_y = r_elbow_rot_y / np.linalg.norm(
        r_elbow_rot_y, axis=1, keepdims=True
    )
    r_elbow_rot_y = np.cross(r_elbow_rot_y, -r_elbow_rot_z)
    r_elbow_rot_y = r_elbow_rot_y / np.linalg.norm(
        r_elbow_rot_y, axis=1, keepdims=True
    )

    r_elbow_rot_x = np.cross(r_elbow_rot_y, r_elbow_rot_z)
    r_elbow_rot_x = r_elbow_rot_x / np.linalg.norm(
        r_elbow_rot_x, axis=1, keepdims=True
    )
    r_elbow_rot = np.stack([r_elbow_rot_x, r_elbow_rot_y, r_elbow_rot_z], axis=2)

    r_upperarm_dir = r_forearm - r_upperarm
    r_upperarm_dir = r_upperarm_dir / np.linalg.norm(
        r_upperarm_dir, axis=1, keepdims=True
    )

    # left knee
    l_knee_rot_z = l_calf - l_foot
    l_knee_rot_z = l_knee_rot_z / np.linalg.norm(
        l_knee_rot_z, axis=1, keepdims=True
    )

    l_knee_rot_y = l_calf - l_thigh
    l_knee_rot_y = l_knee_rot_y / np.linalg.norm(
        l_knee_rot_y, axis=1, keepdims=True
    )
    l_knee_rot_y = np.cross(l_knee_rot_z, l_knee_rot_y)
    l_knee_rot_y = l_knee_rot_y / np.linalg.norm(
        l_knee_rot_y, axis=1, keepdims=True
    )

    l_knee_rot_x = np.cross(l_knee_rot_y, l_knee_rot_z)
    l_knee_rot_x = l_knee_rot_x / np.linalg.norm(
        l_knee_rot_x, axis=1, keepdims=True
    )
    l_knee_rot = np.stack([l_knee_rot_x, l_knee_rot_y, l_knee_rot_z], axis=2)

    l_thigh_dir = l_calf - l_thigh
    l_thigh_dir = l_thigh_dir / np.linalg.norm(l_thigh_dir, axis=1, keepdims=True)

    # right knee
    r_knee_rot_z = r_calf - r_foot
    r_knee_rot_z = r_knee_rot_z / np.linalg.norm(
        r_knee_rot_z, axis=1, keepdims=True
    )

    r_knee_rot_y = r_calf - r_thigh
    r_knee_rot_y = r_knee_rot_y / np.linalg.norm(
        r_knee_rot_y, axis=1, keepdims=True
    )
    r_knee_rot_y = np.cross(r_knee_rot_z, r_knee_rot_y)
    r_knee_rot_y = r_knee_rot_y / np.linalg.norm(
        r_knee_rot_y, axis=1, keepdims=True
    )

    r_knee_rot_x = np.cross(r_knee_rot_y, r_knee_rot_z)
    r_knee_rot_x = r_knee_rot_x / np.linalg.norm(
        r_knee_rot_x, axis=1, keepdims=True
    )
    r_knee_rot = np.stack([r_knee_rot_x, r_knee_rot_y, r_knee_rot_z], axis=2)

    r_thigh_dir = r_calf - r_thigh
    r_thigh_dir = r_thigh_dir / np.linalg.norm(r_thigh_dir, axis=1, keepdims=True)

    # to pybullet
    upper_rot = R.from_matrix(upper_rot)
    lower_rot = R.from_matrix(lower_rot)
    l_elbow_rot = R.from_matrix(l_elbow_rot)
    r_elbow_rot = R.from_matrix(r_elbow_rot)
    l_knee_rot = R.from_matrix(l_knee_rot)
    r_knee_rot = R.from_matrix(r_knee_rot)

    # waist angle
    R_u_l = np.einsum('ijk,ikl->ijl', np.transpose(lower_rot.as_matrix(), (0, 2, 1)), upper_rot.as_matrix())
    R_u_l = R.from_matrix(R_u_l)
    waist_angles = R_u_l.as_euler('xyz', degrees=False)

    # ankle angle
    l_calf_dir = l_calf - l_foot
    l_foot_dir = l_toe - l_foot
    l_foot_dir = l_foot_dir / np.linalg.norm(l_foot_dir, axis=1, keepdims=True)
    cos = np.einsum('ij,ij->i', l_calf_dir, l_foot_dir) / (np.linalg.norm(l_calf_dir, axis=1) * np.linalg.norm(l_foot_dir, axis=1))
    l_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2

    r_calf_dir = r_calf - r_foot
    r_foot_dir = r_toe - r_foot
    r_foot_dir = r_foot_dir / np.linalg.norm(r_foot_dir, axis=1, keepdims=True)
    cos = np.einsum('ij,ij->i', r_calf_dir, r_foot_dir) / (np.linalg.norm(r_calf_dir, axis=1) * np.linalg.norm(r_foot_dir, axis=1))
    r_foot_angle = np.arccos(cos) - np.arctan(0.047/0.160) - np.pi/2


    amass_data = {'num_frames': num_frames,
                'pelvis': pelvis,
                'upper_rot': upper_rot,
                'lower_rot': lower_rot,
                'l_elbow_rot': l_elbow_rot,
                'r_elbow_rot': r_elbow_rot,
                'l_knee_rot': l_knee_rot,
                'r_knee_rot': r_knee_rot,
                'l_upperarm_dir': l_upperarm_dir,
                'r_upperarm_dir': r_upperarm_dir,
                'l_thigh_dir': l_thigh_dir,
                'r_thigh_dir': r_thigh_dir,
                'waist_angles': waist_angles,
                'l_foot_angle': l_foot_angle,
                'r_foot_angle': r_foot_angle,
                'l_foot_dir': l_foot_dir,
                'r_foot_dir': r_foot_dir,
    }

    return amass_data

In [7]:
result = joblib.load('/home/jianrenw/mocap/data/out/amass.pt')

In [8]:
keys = list(result.keys())
current_key = keys[15]

In [9]:
pose = result[current_key]['pose']
mocap_rate = result[current_key]['mocap_framerate']

In [28]:
amass_data = amass2adam(pose)

In [4]:
urdf_path = '/home/jianrenw/mocap/adam/urdf/adam_v2.urdf'

In [25]:
import plotly
import plotly.graph_objs as go

In [26]:
frame_idx = 310
joints = go.Scatter3d(
    x=pose[frame_idx, :, 0],  
    y=pose[frame_idx, :, 1],  
    z=pose[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene = dict(
            xaxis = dict(title="x", range = [-2,0]),
            yaxis = dict(title="y", range = [-1,1]),
            zaxis = dict(title="z", range = [0,2]),
            aspectratio=dict(x=1, y=1, z=1)
    ))

fig = go.Figure(data=[joints], layout=layout)
plotly.offline.iplot(fig)

In [17]:
lower_rot[i].as_matrix()

array([[-0.93095705, -0.33178577, -0.15243746],
       [ 0.33879245, -0.94060894, -0.02178306],
       [-0.13615673, -0.07192375,  0.98807303]])

In [19]:
useful_poses = pose

In [20]:
pelvis = useful_poses[:,0,:]

l_upperarm = useful_poses[:,16,:]
l_forearm = useful_poses[:,18,:]
l_hand = useful_poses[:,20,:]

r_upperarm = useful_poses[:,17,:]
r_forearm = useful_poses[:,19,:]
r_hand = useful_poses[:,21,:]

l_thigh = useful_poses[:,1,:]
l_calf = useful_poses[:,4,:]
l_foot = useful_poses[:,7,:]
l_toe = useful_poses[:,10,:]

r_thigh = useful_poses[:,2,:]
r_calf = useful_poses[:,5,:]
r_foot = useful_poses[:,8,:]
r_toe = useful_poses[:,11,:]

In [84]:
a = pose[frame_idx, 5] - pose[frame_idx, 8]
b = pose[frame_idx, 11] - pose[frame_idx, 8]
c = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
angle = np.arccos(c)
angle * 180 / np.pi

82.99478973698383

In [83]:
pose[frame_idx, 11]

array([-1.0301911 , -0.41919518,  0.01949066], dtype=float32)

In [29]:
physicsClient = p.connect(p.GUI)  # non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])
humanoid = p.loadURDF(urdf_path, robot_start_pos, robot_start_orientation)

num_frames = amass_data['num_frames']
pelvis = amass_data['pelvis']
upper_rot = amass_data['upper_rot']
lower_rot = amass_data['lower_rot']
l_elbow_rot = amass_data['l_elbow_rot']
r_elbow_rot = amass_data['r_elbow_rot']
l_knee_rot = amass_data['l_knee_rot']
r_knee_rot = amass_data['r_knee_rot']
l_upperarm_dir = amass_data['l_upperarm_dir']
r_upperarm_dir = amass_data['r_upperarm_dir']
l_thigh_dir = amass_data['l_thigh_dir']
r_thigh_dir = amass_data['r_thigh_dir']
waist_angles = amass_data['waist_angles']
l_foot_angle = amass_data['l_foot_angle']
r_foot_angle = amass_data['r_foot_angle']
l_foot_dir = amass_data['l_foot_dir']
r_foot_dir = amass_data['r_foot_dir']

upperarm_len = 0.2648365892539233
thigh_len = 0.4252746432372357
toe_len = 0.167
heel_len = 0.079

toe_R = R.from_euler('y', np.arctan(0.047/0.160)).as_matrix()
heel_R = R.from_euler('y', np.pi / 2 + np.arctan(0.047/0.064)).as_matrix()

num_joints = p.getNumJoints(humanoid)

joint_names = []

pelvis_copy = pelvis.copy()

# Iterate through the joints and print their names
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(humanoid, joint_index)
    joint_name = joint_info[1].decode(
        "utf-8"
    )  # Decode the byte string to get the joint name
    joint_names.append(joint_name)

joint_poses = []
jointDamping = [0.1] * num_joints
jointDamping[12] = 100
jointDamping[13] = 100
jointDamping[14] = 100
restPoses = [0.0] * num_joints
init_pose = [0.0] * num_joints

for i in range(num_frames):

    init_pose[12] = waist_angles[i,0]
    init_pose[13] = waist_angles[i,1]
    init_pose[14] = waist_angles[i,2]
    restPoses[12] = waist_angles[i,0]
    restPoses[13] = waist_angles[i,1]
    restPoses[14] = waist_angles[i,2]

    p.resetBasePositionAndOrientation(
        humanoid, (pelvis[i]).tolist(), lower_rot[i].as_quat().tolist()
    )
    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("shoulderRoll_Left")
    )[4:6]
    r_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("shoulderRoll_Right")
    )[4:6]
    l_hip_pos, _ = p.getLinkState(
        humanoid, joint_names.index("hipRoll_Left")
    )[4:6]
    r_hip_pos, _ = p.getLinkState(
        humanoid, joint_names.index("hipRoll_Right")
    )[4:6]

    # IK for l hand
    l_elbow_pos = np.array(l_shoulder_pos) + l_upperarm_dir[i] * upperarm_len

    # IK for r hand
    r_elbow_pos = np.array(r_shoulder_pos) + r_upperarm_dir[i] * upperarm_len

    # IK for l foot
    l_knee_pos = np.array(l_hip_pos) + l_thigh_dir[i] * thigh_len

    # IK for r foot
    r_knee_pos = np.array(r_hip_pos) + r_thigh_dir[i] * thigh_len

    # # Calculate the IK solution
    ik_solution_rh = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("elbow_Right"),
        r_elbow_pos,
        targetOrientation=r_elbow_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed,
        restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lh = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("elbow_Left"),
        l_elbow_pos,
        targetOrientation=l_elbow_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint anglesss
    )

    ik_solution_rf = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("kneePitch_Right"),
        r_knee_pos,
        targetOrientation=r_knee_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint angles
    )

    ik_solution_lf = p.calculateInverseKinematics(
        humanoid,
        joint_names.index("kneePitch_Left"),
        l_knee_pos,
        targetOrientation=l_knee_rot.as_quat()[i],
        jointDamping=jointDamping,  # Adjust damping as needed
        restPoses=restPoses,  # Initial joint angles
    )

    # p.resetJointState(humanoid, joint_index, ik_solution)

    ik_solution = np.zeros(num_joints)
    ik_solution[:4] = ik_solution_lf[:4]
    ik_solution[4] = l_foot_angle[i]
    ik_solution[6:10] = ik_solution_rf[6:10]
    ik_solution[10] = r_foot_angle[i]
    ik_solution[12] = waist_angles[i,0]
    ik_solution[13] = waist_angles[i,1]
    ik_solution[14] = waist_angles[i,2]
    ik_solution[16:20] = ik_solution_lh[15:19]
    ik_solution[24:28] = ik_solution_rh[19:23]

    restPoses = ik_solution
    init_pose = ik_solution

    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)

    l_ankle_pos, l_ankle_rot = p.getLinkState(
        humanoid, joint_names.index("anklePitch_Left")
    )[4:6]
    r_ankle_pos, r_ankle_rot = p.getLinkState(
        humanoid, joint_names.index("anklePitch_Right")
    )[4:6]

    l_rot_matrix = p.getMatrixFromQuaternion(l_ankle_rot)
    l_rot_matrix = np.array(l_rot_matrix).reshape(3, 3)
    l_toe_pose = np.dot(l_rot_matrix@toe_R, [toe_len, 0, 0]) + l_ankle_pos
    l_heel_pose = np.dot(l_rot_matrix@heel_R, [heel_len, 0, 0]) + l_ankle_pos

    r_rot_matrix = p.getMatrixFromQuaternion(r_ankle_rot)
    r_rot_matrix = np.array(r_rot_matrix).reshape(3, 3)
    r_toe_pose = np.dot(r_rot_matrix@toe_R, [toe_len, 0, 0]) + r_ankle_pos
    r_heel_pose = np.dot(r_rot_matrix@heel_R, [heel_len, 0, 0]) + r_ankle_pos

    sphere_radius = 0.05
    visual_shape_id = p.createVisualShape(p.GEOM_SPHERE, radius=sphere_radius, rgbaColor=[1, 0, 0, 1])  # Red sphere
    sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=l_toe_pose)
    sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=l_heel_pose)
    sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_toe_pose)
    sphere_body_id = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visual_shape_id, basePosition=r_heel_pose)
    if i == 0:
        min_foot_z = np.min([l_toe_pose[2], r_toe_pose[2], l_heel_pose[2], r_heel_pose[2]])

    pelvis_copy[i, 2] = pelvis[i,2] - min_foot_z

    # Step the simulation
    joint_poses.append(ik_solution)
    time.sleep(0.01)

p.disconnect()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.29.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.29.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
dest

In [86]:
r_foot_angle[i,0] * 180 / np.pi

-30.33916070825221

In [6]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [85]:
np.arctan(0.047/0.160) * 180 / np.pi

16.37015090008355

In [87]:
np.arccos(-0.5)

2.0943951023931957